# Xây dựng mô hình từ giải thuật SVM trên dữ liệu bệnh tiểu đường.

#### Tải thư viện và nạp dữ liệu:

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('datasets/diabetes.csv')

#### Xử lý dữ liệu thiếu

In [3]:
cols_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in cols_to_clean:
    # Lấy giá trị trung bình (chỉ tính các giá trị không phải 0)
    col_mean = df[col][df[col] != 0].mean()
    # Thay thế các giá trị 0 bằng giá trị trung bình
    df[col] = df[col].replace(0, col_mean)

#### Chuẩn hóa dữ liệu (Scaling)

In [4]:
from sklearn.preprocessing import StandardScaler

# Tách X (features) và y (target)
target = 'Outcome'
features = df.drop(target, axis=1).columns.tolist()

X = df[features]
y = df[target]

# Khởi tạo StandardScaler
scaler = StandardScaler()

# Fit và transform X
X_scaled = scaler.fit_transform(X)

#### Chia dữ liệu

In [ ]:
# Tách dữ liệu đã chuẩn hóa
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.2,       
    random_state=101     
)

#### Xây dựng và Tối ưu SVM

In [6]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# 1. Khởi tạo mô hình
svc_model = svm.SVC()

# 2. Tạo lưới tham số để tìm kiếm
# 'C' là tham số điều chỉnh lề mềm 
# 'kernel' là các hàm nhân (linear, rbf) 
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly']
}

# 3. Khởi tạo GridSearchCV
# cv=4 nghĩa là 4-fold cross-validation 
grid = GridSearchCV(svc_model, param_grid, refit=True, verbose=2, cv=4, scoring='accuracy')

# 4. Huấn luyện mô hình
grid.fit(X_train, y_train)

# 5. In ra kết quả tốt nhất
print("Tham số tốt nhất tìm được:")
print(grid.best_params_)

# Lấy ra mô hình tốt nhất
best_svm = grid.best_estimator_

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.0s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.0s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.0s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.0s
[CV] END .................................C=0.1, kernel=poly; total time=   0.0s
[CV] END .................................C=0.1, kernel=poly; total time=   0.0s
[CV] END .................................C=0.1, kernel=poly; total time=   0.0s
[CV] END .................................C=0.1,

##### Đánh giá mô hình

In [8]:
from sklearn.metrics import classification_report, accuracy_score

# Dự đoán trên tập test
predictions = best_svm.predict(X_test)

# In kết quả
print("\n--- Kết quả đánh giá trên tập Test ---")
print(f"Độ chính xác (Accuracy): {accuracy_score(y_test, predictions):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, predictions))


--- Kết quả đánh giá trên tập Test ---
Độ chính xác (Accuracy): 0.7792

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       103
           1       0.71      0.57      0.63        51

    accuracy                           0.78       154
   macro avg       0.76      0.73      0.74       154
weighted avg       0.77      0.78      0.77       154



Kết thúc